In [1]:
import ee
import geemap
import geedim as gd
import geopandas as gpd

In [2]:
geemap.set_proxy(port=4780)

In [4]:
ee.Authenticate()

Enter verification code: 4/1AbUR2VP5DnVRfpWIW2ANHHAmhyDBR76F50PwnE25ZJjDIPbS9lKAgtJJmNE

Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

In [7]:
geometry_dir = "/Users/lilywang/Desktop/phD/Qilian/UAV-LULC/downloaded-datasets/Suli/suli.shp"
region = geemap.shp_to_ee(geometry_dir).geometry()

In [8]:
def clip_to_shapefile(img):
    return img.clip(region)

In [9]:
def calculate_ndvi(img):
    NDVI = img.normalizedDifference(['B8','B4'])
    return NDVI

In [10]:
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

In [11]:
def get_s2_sr_cld_col(month,aoi):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filter(ee.Filter.calendarRange(month,month,'month'))
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
        .map(clip_to_shapefile))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filter(ee.Filter.calendarRange(month,month,'month'))
        .map(clip_to_shapefile))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [12]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [13]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [14]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [15]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [16]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [17]:
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = region.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

In [18]:
s2_sr_cld_col_eval1 = get_s2_sr_cld_col(1,region)

In [19]:
s2_sr_median1 = (s2_sr_cld_col_eval1.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [20]:
s2_sr_cld_col_eval2 = get_s2_sr_cld_col(2,region)
s2_sr_median2 = (s2_sr_cld_col_eval2.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [21]:
s2_sr_cld_col_eval3 = get_s2_sr_cld_col(3,region)
s2_sr_median3 = (s2_sr_cld_col_eval3.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [22]:
s2_sr_cld_col_eval4 = get_s2_sr_cld_col(4,region)
s2_sr_median4 = (s2_sr_cld_col_eval4.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [23]:
s2_sr_cld_col_eval5 = get_s2_sr_cld_col(5,region)
s2_sr_median5 = (s2_sr_cld_col_eval5.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [24]:
s2_sr_cld_col_eval6 = get_s2_sr_cld_col(6,region)
s2_sr_median6 = (s2_sr_cld_col_eval6.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [25]:
s2_sr_cld_col_eval7 = get_s2_sr_cld_col(7,region)
s2_sr_median7 = (s2_sr_cld_col_eval7.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [26]:
s2_sr_cld_col_eval8 = get_s2_sr_cld_col(8,region)
s2_sr_median8 = (s2_sr_cld_col_eval8.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [27]:
s2_sr_cld_col_eval9 = get_s2_sr_cld_col(9,region)
s2_sr_median9 = (s2_sr_cld_col_eval9.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [28]:
s2_sr_cld_col_eval10 = get_s2_sr_cld_col(10,region)
s2_sr_median10 = (s2_sr_cld_col_eval10.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [29]:
s2_sr_cld_col_eval11 = get_s2_sr_cld_col(11,region)
s2_sr_median11 = (s2_sr_cld_col_eval11.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [30]:
s2_sr_cld_col_eval12 = get_s2_sr_cld_col(12,region)
s2_sr_median12 = (s2_sr_cld_col_eval12.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [29]:
# Create a folium map object.
center = region.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m.add_ee_layer(s2_sr_median7,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [31]:
cloudfree = ee.ImageCollection([s2_sr_median1,s2_sr_median2,s2_sr_median3,s2_sr_median4,
                                s2_sr_median5,s2_sr_median6,s2_sr_median7,s2_sr_median8,
                                s2_sr_median9,s2_sr_median10,s2_sr_median11,s2_sr_median12]).map(calculate_ndvi)

In [34]:
s2_sr_median1.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B6',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min

In [32]:
geemap.ee_export_image_to_drive(
    s2_sr_median1,
    crs = "EPSG:4326",folder = "Month01",
    region = region.bounds(), scale=10,maxPixels=99999999999
)

In [33]:
geemap.ee_export_image_to_drive(
    s2_sr_median2,
    crs = "EPSG:4326",folder = "Month02",
    region = region.bounds(), scale=10,maxPixels=99999999999
)

In [34]:
geemap.ee_export_image_to_drive(
    s2_sr_median3,
    crs = "EPSG:4326",folder = "Month03",
    region = region.bounds(), scale=10,maxPixels=99999999999
)

In [35]:
geemap.ee_export_image_to_drive(
    s2_sr_median4,
    crs = "EPSG:4326",folder = "Month04",
    region = region.bounds(), scale=10,maxPixels=99999999999
)

In [36]:
geemap.ee_export_image_to_drive(
    s2_sr_median5,
    crs = "EPSG:4326",folder = "Month05",
    region = region.bounds(), scale=10,maxPixels=99999999999
)

In [37]:
geemap.ee_export_image_to_drive(
    s2_sr_median6,
    crs = "EPSG:4326",folder = "Month06",
    region = region.bounds(), scale=10,maxPixels=99999999999
)

In [38]:
geemap.ee_export_image_to_drive(
    s2_sr_median7,
    crs = "EPSG:4326",folder = "Month07",
    region = region.bounds(), scale=10,maxPixels=99999999999
)

In [39]:
geemap.ee_export_image_to_drive(
    s2_sr_median8,
    crs = "EPSG:4326",folder = "Month08",
    region = region.bounds(), scale=10,maxPixels=99999999999
)

In [40]:
geemap.ee_export_image_to_drive(
    s2_sr_median9,
    crs = "EPSG:4326",folder = "Month09",
    region = region.bounds(), scale=10,maxPixels=99999999999
)

In [41]:
geemap.ee_export_image_to_drive(
    s2_sr_median10,
    crs = "EPSG:4326",folder = "Month10",
    region = region.bounds(), scale=10,maxPixels=99999999999
)

In [42]:
geemap.ee_export_image_to_drive(
    s2_sr_median11,
    crs = "EPSG:4326",folder = "Month11",
    region = region.bounds(), scale=10,maxPixels=99999999999
)

In [43]:
geemap.ee_export_image_to_drive(
    s2_sr_median12,
    crs = "EPSG:4326",folder = "Month12",
    region = region.bounds(), scale=10,maxPixels=99999999999
)